**Add your LiU-ID here:**
* <liuid 1>
* <liuid 2>

### **0. Quick introduction to jupyter notebooks**
* Each cell in this notebook contains either code or text.
* You can run a cell by pressing Ctrl-Enter, or run and advance to the next cell with Shift-Enter.
* Code cells will print their output, including images, below the cell. Running it again deletes the previous output, so be careful if you want to save some reuslts.
* You don't have to rerun all cells to test changes, just rerun the cell you have made changes to. Some exceptions might apply, for example if you overwrite variables from previous cells, but in general this will work.
* If all else fails, use the "Kernel" menu and select "Restart Kernel and Clear All Output". You can also use this menu to run all cells.

### **0.5 Setup**

<span style="background-color: #ffb8b8ff; padding: 5px; border-radius: 5px; font-weight: bold; color: #800000ff; font-size: 1.2em;">
This notebook is entirely optional, and only meant for those who want to dive deeper.
</span>

In [ ]:
# Automatically reload modules when changed
%reload_ext autoreload
%autoreload 2
# Plot figures "inline" with other output
%matplotlib inline

# Import modules, classes, functions
from matplotlib import pyplot as plt
import numpy as np

from utils import loadDataset, splitData, plotTrainingProgress, plotResultsDots, plotConfusionMatrixOCR
from evalFunctions import calcAccuracy, calcConfusionMatrix

# Configure nice figures
plt.rcParams['figure.facecolor']='white'
plt.rcParams['figure.figsize']=(8, 5)

### **1. N-layer neural network**

You will now implement a more general N-layer neural network, where you can freely choose how many layers to use for each model. This means that the inputs and outputs from the functions, such as the weights and biases, are passed as lists. The structure of the code is otherwise still the same, with a `forward`, `backward`, and `update` function.

#### **1.1 Implementing the forward pass**

This implementation will look similar to the previous tasks, except most inputs and outputs are now lists of matrices. Additionally, the previous input parameter "useTanhOutput" is removed, and instead the activation function for each layer is passed as a list of strings. You can implement support for different activation functions, such as "tanh", "relu", "lrelu" (leaky ReLU), and "linear". Note that the activation function for the output layer is also specified in this list.

In [ ]:
def forward(X, W, B, activations):
    """Forward pass of two layer network

    Args:
        X (array): Input samples.
        W (list of arrays): Neural network weights for each layer.
        B (list of arrays): Neural network biases for each layer.
        activations (list of str): Activation functions for each layer, including output layer.
            You can support whichever you want, recommended 'linear', 'tanh', and 'relu'.

    Returns:
        Y (array): Output for each sample and class.
        L (array): Resulting label of each sample.
        U (list of arrays): Output of each hidden layer.
    """

    # --------------------------------------------
    # === Your code here =========================
    # --------------------------------------------
    
    # Initialize list to store outputs of each layer
    U = [None] * len(W)

    # Initialize a working variable to hold the current value of
    # Y as we propagate through the network
    YTmp = ???

    # Loop through hidden layers
    for i = ???
        # Calculate layer output based on previous layer output
        YTmp = ???

        # Apply activation function for current layer
        if activations[i] == 'linear':
            YTmp = ???
        elif activations[i] == 'tanh':
            YTmp = ???
        # Etc...
        
        # Store output of current layer, including output layer
        U[i] = YTmp

    # ============================================
    
    Y = YTmp

    # Calculate labels
    L = Y.argmax(axis=1)

    return Y, L, U

#### **1.2 Implementing the backward pass**

To implement the backward pass, you will again need to loop over all layers, this time in reverse order. Since we do not know exlicitly the number of layers, you will have to implement the gradient calculation iteratively instead of unrolling it for each layer, like we did in the two-layer network.

In [ ]:
def backward(W, B, X, U, Y, D, activations):
    """Compute the gradients for network weights and biases

    Args:
        W (list of arrays): Current values of the network weights for each layer.
        B (list of arrays): Current values of the network biases for each layer.
        X (array): Training samples.
        U (list of arrays): Intermediate outputs of the hidden layers.
        Y (array): Predicted outputs.
        D (array): Target outputs.
        activations (list of str): Activation functions for each layer, including output layer.
            You can support whichever you want, recommended 'linear', 'tanh', and 'relu'.

    Returns:
        GradW (list of arrays): Gradients with respect to the network weights for each layer.
        GradB (list of arrays): Gradients with respect to the network biases for each layer.
    """
    
    N  = Y.shape[0]
    NC = Y.shape[1]
    
    # --------------------------------------------
    # === Your code here =========================
    # --------------------------------------------

    # Initialize lists to store gradients
    GradW = [None] * len(W)
    GradB = [None] * len(B)

    # Compute initial delta (error) with scaling
    delta = ???

    # Loop backwards through layers
    for i in reversed(range(len(W))):
        # Get input to current layer
        In = ???

        # Apply derivative of activation function
        if activations[i] == "tanh":
            delta *= ???
        elif activations[i] == "relu":
            delta *= ???
        # Etc...

        # Compute gradients for weights and biases
        GradW[i] = ???
        GradB[i] = ???

        # Update delta for next layer (if not input layer)
        if i > 0:
            delta = ???
    
    # ============================================
    
    return GradW, GradB

#### **1.3 Implementing the weight update**

In [ ]:
def update(W, B, GradW, GradB, params):
    """Update weights and biases using computed gradients.

    Args:
        W (list of arrays): Current values of the network weights for each layer.
        B (list of arrays): Current values of the network biases for each layer.
        
        GradW (list of arrays): Gradients with respect to the network weights for each layer.
        GradB (list of arrays): Gradients with respect to the network biases for each layer.
        
        params (dict):
            - learningRate: Scale factor for update step.
            - momentum: Scale factor for momentum update (optional).
        
    Returns:
        W (list of arrays): Updated network weights for each layer.
        B (list of arrays): Updated network biases for each layer.
    """
    
    LR = params["learningRate"]
    
    # For optional task on momentum
    M = params["momentum"]
    PrevGradW = params["PrevGradW"]
    PrevGradB = params["PrevGradB"]
    
    # --------------------------------------------
    # === Your code here =========================
    # --------------------------------------------
    
    # Tip, use list comprehensions to compute the updates in a concise way
    
    TotGradW = ???
    TotGradB = ???
    
    W = ???
    B = ???
    
    # ============================================
    
    # For optinal task on momentum
    params["PrevGradW"] = TotGradW
    params["PrevGradB"] = TotGradB
    
    return W, B

#### **1.4 The training function**

In [ ]:
def trainMultiLayer(XTrain, DTrain, XTest, DTest, W0, B0, activations, params):
    """Trains a N-layer network

    Args:
        XTrain (array): Training samples.
        DTrain (array): Training network target values.
        XTest (array): Test samples.
        DTest (array): Test network target values.
        W0 (list of arrays): Initial values of the network weights for each layer.
        B0 (list of arrays): Initial values of the network biases for each layer.
        activations (list of str): Activation functions for each layer, including output layer.
        params (dict): Dictionary containing:
            epochs (int): Number of training steps.
            learningRate (float): Size of a training step.

    Returns:
        W (list of arrays): Network weights after training.
        B (list of arrays): Network biases after training.
        metrics (dict): Losses and accuracies for training and test data.
    """

    # Initialize variables
    metrics = {keys:np.zeros(params["epochs"]+1) for keys in ["lossTrain", "lossTest", "accTrain", "accTest"]}

    # Set default activations if none provided
    if activations is None:
        activations = ["tanh"] * (len(W0) - 1) + ["linear"]

    if "momentum" not in params:
        params["momentum"] = 0

    nTrain = XTrain.shape[0]
    nTest  = XTest.shape[0]
    nClasses = DTrain.shape[1]
    
    # For optinal task on momentum
    params["PrevGradW"] = [np.zeros_like(W) for W in W0]
    params["PrevGradB"] = [np.zeros_like(B) for B in B0]
    
    # Set initial weights and biases
    W = W0
    B = B0

    # Get class labels
    LTrain = np.argmax(DTrain, axis=1)
    LTest  = np.argmax(DTest , axis=1)

    # Calculate initial metrics
    YTrain, LTrainPred, UTrain = forward(XTrain, W, B, activations)
    YTest , LTestPred , _      = forward(XTest , W, B, activations)
    
    # Including the initial metrics makes the progress plots worse, set nan to exclude
    metrics["lossTrain"][0] = np.nan # ((YTrain - DTrain)**2).mean()
    metrics["lossTest"][0]  = np.nan # ((YTest  - DTest )**2).mean()
    metrics["accTrain"][0]  = np.nan # (LTrainPred == LTrain).mean()
    metrics["accTest"][0]   = np.nan # (LTestPred  == LTest ).mean()

    # Create figure for plotting progress
    fig = plt.figure(figsize=(20,8), tight_layout=True)

    # Training loop
    for n in range(1, params["epochs"]+1):
        
        # Compute gradients...
        GradW, GradB = backward(W, B, XTrain, UTrain, YTrain, DTrain, activations)
        # ... and update weights
        W, B = update(W, B, GradW, GradB, params)
        
        # Evaluate errors
        YTrain, LTrainPred, UTrain = forward(XTrain, W, B, activations)
        YTest , LTestPred , _      = forward(XTest , W, B, activations)
        metrics["lossTrain"][n] = ((YTrain - DTrain)**2).mean()
        metrics["lossTest"][n]  = ((YTest  - DTest )**2).mean()
        metrics["accTrain"][n]  = (LTrainPred == LTrain).mean()
        metrics["accTest"][n]   = (LTestPred  == LTest ).mean()

        # Plot progress
        if (not n % (params["epochs"] // 25)) or (n == params["epochs"]):
            plotMode = "network" if W[0].shape[0] < 64 else "ocr"
            plotTrainingProgress(fig, W, B, metrics, n=n, cmap='coolwarm', mode=plotMode)

    return W, B, metrics

We also define the same function for normalizing the data, which is even more important now that we have more than one layer.

In [ ]:
def normalize(X, XRef):
    """
    Normalizes the data X with the mean and standard deviation of the reference data XRef. These can be the same dataset.
    
    Args:
       X (array): Data matrix to be normalized, features are in axis 0.
       XRef (array): Data matrix for calculating the normalization parameters, features are in axis 0.
       
    Returns:
       X (array): Input X normalized with XRef parameters.
    """
    # Compute mean and std of the reference data set
    m = XRef.mean(axis=0)
    s = XRef.std(axis=0)
    # Prevent division by 0 is feature has no variance
    s[s == 0] = 1
    # Return normalized data
    return (X - m) / s

---
### **2 Optimizing each dataset**

Like before, we define a function that performs the steps for training the networks.

In [ ]:
def  trainMultiLayerOnDataset(datasetNr, testSplit, W0, B0, activations, params):
    """Train a multi layer network on a specific dataset.

    Ags:
        datasetNr (int): ID of dataset to use
        testSplit (float): Fraction of data reserved for testing.
        W0 (list of arrays): Initial values of the network weights for each layer.
        B0 (list of arrays): Initial values of the network biases for each layer.
        activations (list of str): Activation functions for each layer, including output layer.
        params (dict): Dictionary containing:
            epochs (int): Number of training steps.
            learningRate (float): Size of a training step.
    """
    
    # Load data and split into training and test sets
    X, D, L = loadDataset(datasetNr)
    XTrain, DTrain, LTrain, XTest, DTest, LTest = splitData(X, D, L, testSplit)
    
    if "normalize" in params and params["normalize"]:
        XTrainNorm = normalize(XTrain, XTrain)
        XTestNorm  = normalize(XTest, XTrain)
    else:
        XTrainNorm = XTrain
        XTestNorm  = XTest

    # Train network
    W, B, metrics = trainMultiLayer(XTrainNorm, DTrain, XTestNorm, DTest, W0, B0, activations, params)

    # Predict classes on test set
    LPredTrain = forward(XTrainNorm, W, B, activations)[1]
    LPredTest  = forward(XTestNorm , W, B, activations)[1]

    # Compute metrics
    accTrain = calcAccuracy(LPredTrain, LTrain)
    accTest  = calcAccuracy(LPredTest , LTest)
    confMatrix = calcConfusionMatrix(LPredTest, LTest)

    # Display results
    print(f'Train accuracy: {accTrain:.4f}')
    print(f'Test accuracy: {accTest:.4f}')
    print("Test data confusion matrix:")
    print(confMatrix)

    if datasetNr < 4:
        plotResultsDots(XTrainNorm, LTrain, LPredTrain, XTestNorm, LTest, LPredTest, lambda X: forward(X, W, B, activations)[1])
    else:
        plotConfusionMatrixOCR(XTest, LTest, LPredTest)

#### **2.1 Optimizing dataset 1**

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

nInputs = ???
nClasses = ???
nHidden = [???, ???]

# Weights are expected to be lists of np.arrays
W0 = [???, ???]
B0 = [???, ???]
      
# Activations are expected to be a list of strings with activation function names, including output activation
activations = ["???", "???", "???"]

params = {"epochs": 2000, "learningRate": 0.05, "normalize": True, "momentum": 0.9}

# ============================================

trainMultiLayerOnDataset(1, 0.15, W0, B0, activations, params)

#### **2.2 Optimizing dataset 2**

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

nInputs = ???
nClasses = ???
nHidden = [???, ???]

# Weights are expected to be lists of np.arrays
W0 = [???, ???]
B0 = [???, ???]
      
# Activations are expected to be a list of strings with activation function names, including output activation
activations = ["???", "???", "???"]

params = {"epochs": 2000, "learningRate": 0.05, "normalize": True, "momentum": 0.9}

# ============================================

trainMultiLayerOnDataset(2, 0.15, W0, B0, activations, params)

#### **2.3 Optimizing dataset 3**

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

nInputs = ???
nClasses = ???
nHidden = [???, ???]

# Weights are expected to be lists of np.arrays
W0 = [???, ???]
B0 = [???, ???]
      
# Activations are expected to be a list of strings with activation function names, including output activation
activations = ["???", "???", "???"]

params = {"epochs": 2000, "learningRate": 0.05, "normalize": True, "momentum": 0.9}

# ============================================

trainMultiLayerOnDataset(3, 0.15, W0, B0, activations, params)

#### **2.4 Optimizing dataset 4**

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

nInputs = ???
nClasses = ???
nHidden = [???, ???]

# Weights are expected to be lists of np.arrays
W0 = [???, ???]
B0 = [???, ???]
      
# Activations are expected to be a list of strings with activation function names, including output activation
activations = ["???", "???", "???"]

params = {"epochs": 2000, "learningRate": 0.05, "normalize": True, "momentum": 0.9}

# ============================================

trainMultiLayerOnDataset(4, 0.15, W0, B0, activations, params)